In [33]:
import pandas as pd
import numpy as np
log = pd.read_csv("./00_log.csv", header = None)
users = pd.read_csv("./00_users.csv", encoding="koi8-r", sep="\t")
log.columns = ["user_id","time","bet","win"]
users.columns = ["user_id","mail","address"]

log = log[log.user_id != '#error']
log.user_id = log.user_id.str.split(' - ').apply(lambda x: x[1])
users.user_id = users.user_id.apply(lambda x: x.lower())  

In [34]:
# преобразование времени
log.time = log.time.apply(lambda s: s[1:] if type(s) == str else s)
log.time = pd.to_datetime(log.time)


In [35]:
log.bet.fillna(0, inplace=True)

In [36]:
def fillna_win(row):
    if not np.isnan(row.win):
        return row.win
    elif row.bet == 0:
        return 0
    elif row.bet != 0:
        return -row.bet
    else:
        return ""

new_win = log.apply(lambda row: fillna_win(row), axis=1)
log.win = new_win

In [37]:
def net(row):
    if row.win < 0:
        return row.win
    else:
        return row.win - row.bet

log["net"] = log.apply(lambda row: net(row), axis=1)

Объедините датасеты log.csv и users.csv по признаку user_id по примеру выше. Ответ запишите в поле ниже.



In [38]:
df = pd.merge(log, users, on="user_id")  

In [40]:
df

,user_id,time,bet,win,net,mail,address
0,user_919,2019-01-01 14:06:51,0.0,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
1,user_919,2019-01-30 10:06:00,0.0,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
2,user_919,2019-02-05 14:33:44,0.0,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
3,user_919,2019-02-14 11:38:05,0.0,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
4,user_919,2019-03-02 04:23:36,300.0,-300.0,-300.0,Chikkaverle@icloud.com,Хабаровск
...,...,...,...,...,...,...,...
970,user_932,2019-02-24 22:40:06,0.0,0.0,0.0,BraceWalker@bk.ru,Красноярск
971,user_932,2019-03-15 10:56:14,0.0,0.0,0.0,BraceWalker@bk.ru,Красноярск
972,user_932,2019-03-18 10:13:24,0.0,0.0,0.0,BraceWalker@bk.ru,Красноярск
973,user_932,2019-03-27 12:18:24,0.0,0.0,0.0,BraceWalker@bk.ru,Красноярск


В данном случае мы группируем данные по признаку user_id.

После этого мы в каждой группе выбираем признак win.

Затем мы берем медиану каждой группы по признаку win и на выходе получаем таблицу, в которой индексом является признак user_id. В этой таблице единственный столбец - медиана по каждой группе (то есть по каждому пользователю).

Наконец, последний вызов median() дает нам медиану по предыдущему столбцу, то есть возвращает одно число.

In [44]:
df.groupby('user_id').win.median().median()

0.0

Посчитайте медиану баланса по каждому пользователю. Для этого сгруппируйте по пользователям, возьмите признак net, просуммируйте по каждому пользователю и получите медиану.


In [42]:
df.groupby('user_id').net.sum().median()  

1986.0

Сколько раз в среднем каждый человек приходит, не делая ставок, при условии, что у этого человека все-таки есть хотя бы одна ставка? Например: Человек посетил букмекерскую контору 5 раз из них 1 раз сделал ставку, 4 раза нет - условие выполняется. Человек посетил букмекерскую контору 5 раз из них ни разу ставку не сделал - условие не выполняется. Для того, чтобы узнать это, просуммируйте в каждой группе количество записей со ставкой, равной 0, и поделите на общее количество групп. Если при этом в группе нет записей со ставкой больше 0, считаем количество записей в данной группе равным 0.



In [134]:
def get_user_stat(user):
    s = []
    s.append(user.bet.sum())
    s.append(user.bet.max())
    s.append(user.bet.count())
    counter = 0
    if user.bet.sum() != 0:
        for bet in user.bet:
            if bet == 0:
                counter += 1
    s.append(counter)    
    return pd.Series(
        s, index=[
            ["bet", "bet", "bet", "bet"],
            ["sum", "max", "count", "zero_bet_count"]
        ]
    )


In [139]:
df1 = df.groupby('user_id').apply(get_user_stat)

In [141]:
df1["bet"]["zero_bet_count"].mean()

5.05

In [142]:
for k,i in df.groupby('user_id').bet:
    print(i)

829      0.0
830    800.0
831      0.0
832      0.0
833      0.0
834      0.0
835      0.0
836      0.0
837    100.0
838    400.0
839    300.0
840      0.0
Name: bet, dtype: float64
56      0.0
57      0.0
58      0.0
59      0.0
60      0.0
61      0.0
62    200.0
63    200.0
64    300.0
65    300.0
66    400.0
67    600.0
68    900.0
69    900.0
Name: bet, dtype: float64
512      0.0
513      0.0
514      0.0
515      0.0
516      0.0
517      0.0
518    100.0
519    500.0
Name: bet, dtype: float64
917      0.0
918      0.0
919      0.0
920      0.0
921    200.0
922    300.0
923    400.0
924    800.0
Name: bet, dtype: float64
23        0.0
24        0.0
25        0.0
26        0.0
27      500.0
28      700.0
29      900.0
30     1000.0
31    10454.0
Name: bet, dtype: float64
813      0.0
814      0.0
815      0.0
816      0.0
817      0.0
818    600.0
Name: bet, dtype: float64
388        0.0
389        0.0
390        0.0
391     8700.0
392      950.0
393    12945.0
394    27000.0
395

In [131]:
counter = 0
for bet in df[df.user_id=="user_900"].bet:
    if bet == 0:
        counter += 1
print(counter)

6


In [144]:
df

,user_id,time,bet,win,net,mail,address
0,user_919,2019-01-01 14:06:51,0.0,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
1,user_919,2019-01-30 10:06:00,0.0,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
2,user_919,2019-02-05 14:33:44,0.0,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
3,user_919,2019-02-14 11:38:05,0.0,0.0,0.0,Chikkaverle@icloud.com,Хабаровск
4,user_919,2019-03-02 04:23:36,300.0,-300.0,-300.0,Chikkaverle@icloud.com,Хабаровск
...,...,...,...,...,...,...,...
970,user_932,2019-02-24 22:40:06,0.0,0.0,0.0,BraceWalker@bk.ru,Красноярск
971,user_932,2019-03-15 10:56:14,0.0,0.0,0.0,BraceWalker@bk.ru,Красноярск
972,user_932,2019-03-18 10:13:24,0.0,0.0,0.0,BraceWalker@bk.ru,Красноярск
973,user_932,2019-03-27 12:18:24,0.0,0.0,0.0,BraceWalker@bk.ru,Красноярск


Сколько в среднем времени проходит между появлением человека в сервисе и первой ставкой? Считать нужно только тех, кто делал ставку. Для того, чтобы узнать это, напишите метод, считающий минимальное время среди ставок, равных 0, и минимальное время среди ставок больше 0. После этого верните разницу между вторым и первым числом. Пройдитесь по всем группам. Если в группе нет ставок больше 0, пропустите эту группу. Просуммируйте разницу во времени для каждой группы (с помощью метода, описанного выше) и поделите на количество групп, которые вы не пропустили.


In [173]:
def get_bet_time(user):
    s = []
    min_zero = user[user.bet == 0].time.min()
    min_bet = user[user.bet != 0].time.min()
    delta = min_bet - min_zero
    if delta < pd.Timedelta(0):
        delta = pd.Timedelta(0)
    s.append(min_zero)
    s.append(min_bet)
    s.append(delta)
    return pd.Series(
        s, index=[
            ["time", "time", "time"],
            ["min_zero", "min_bet", "delta"]
        ]
    )


In [175]:
df1 = df.groupby('user_id').apply(get_bet_time)
df1["time"]["delta"].mean()

Timedelta('47 days 01:34:33.930000')

In [166]:
df[df.user_id=="user_996"][df.bet == 0].time.min()

<ipython-input-166-6c108c32b528>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df.user_id=="user_996"][df.bet == 0].time.min()


Timestamp('2019-03-03 11:43:38')

In [157]:
df[df.user_id=="user_996"].time

829   2019-01-21 19:31:34
830   2019-01-28 20:57:37
831   2019-02-10 05:33:25
832   2019-02-17 02:34:32
833   2019-02-21 12:01:29
834   2019-03-02 17:17:13
835   2019-03-03 12:34:16
836   2019-03-23 23:47:07
837   2019-03-27 03:39:36
838   2019-04-05 14:45:54
839   2019-04-05 06:48:33
840   2019-04-06 22:07:35
Name: time, dtype: datetime64[ns]

In [170]:
td = df[df.user_id=="user_996"][df.bet != 0].time.min() - df[df.user_id=="user_996"][df.bet == 0].time.min()

<ipython-input-170-c24619bccf5d>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  td = df[df.user_id=="user_996"][df.bet != 0].time.min() - df[df.user_id=="user_996"][df.bet == 0].time.min()
<ipython-input-170-c24619bccf5d>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  td = df[df.user_id=="user_996"][df.bet != 0].time.min() - df[df.user_id=="user_996"][df.bet == 0].time.min()


In [172]:
if td < pd.Timedelta(0):
    print('s')

s


In [168]:
df[df.user_id=="user_996"][["bet", "time"]]

,bet,time
651,100.0,2019-01-10 07:32:17
652,0.0,2019-03-03 11:43:38
653,0.0,2019-03-09 12:51:58
654,100.0,2019-03-10 10:57:52
655,800.0,2019-04-10 04:42:20


Используйте датасет, который получился в результате всех преобразований в прошлой секции (в том числе, заполнение пропусков).

Ответ запишите в поле ниже в виде одного слова, с большой буквы.

Наибольший суммарный выигрыш среди всех городов имеет Москва. Посчитайте следующий за ней город. Для этого сгруппируйте по городам, возьмите признак win, просуммируйте по каждому городу, отсортируйте и получите второй город.


In [188]:
df.groupby("address").win.sum().sort_values(ascending=False)

address
Москва             11379755.0
Воронеж              107514.0
Санкт-Петербург      105345.0
Казань                75352.0
Ярославль             46845.0
Ижевск                34479.0
Хабаровск             33451.0
Екатеринбург          28582.0
Красноярск            27397.0
Арзангелтск           25813.0
Пермь                 19518.0
Краснодар              3248.0
Ставрополь              487.0
Тюмень                 -499.0
Name: win, dtype: float64

Подсказки:

1. Можно использовать методы min() и max().

2. Учитывайте, что минимальная ставка, это ставка, которая была сделана, т.е. если ставка равна нулю - значит ставки не было.

3. Ответ запишите в поле ниже в виде целого числа (нужно отбросить дробную часть).

Во сколько раз различаются максимальное и минимальное значение средней ставки по городам? Для того, чтобы это посчитать, нужно сгруппировать по городам, взять среднее от признака bet, найти максимальное и минимальное значения, затем поделить одно на другое.


In [196]:
max_ = df[df.bet != 0].groupby("address").bet.mean().max()
min_ = df[df.bet != 0].groupby("address").bet.mean().min()
max_ / min_

127.81602335164835

Давайте повторим все, что мы прошли в этой секции. Напишите код, который посчитает, сколько раз пользователи приходили в букмекерскую контору в каждом городе. Для этого:

1. загрузите датасеты log.csv и users.csv;

2. удалите user_id с ошибкой (#error) и приведите признак user_id к одному виду в обоих датасетах;

3. слейте два датасета в один по признаку user_id;

4. сгруппируйте данные по правильному признаку (какому - вам нужно понять самим), затем выберите user_id и воспользуйтесь функцией count() для подсчета наблюдений в каждой группе;

5. результат запишите в sample2 (объект Series).



In [200]:
type(df.groupby("address").bet.count())

pandas.core.series.Series